In [ ]:
import pandas as pd
from pathlib import Path

wd = '/mnt/tupi/WRI/livestock_global_modeling'

livestock_covs = pd.read_csv(f'{wd}/livestock_census_ard/livestock_cov.csv')
livestock_covs

In [21]:
# Livestock production systems
livestock_covs = pd.concat([
    livestock_covs,
    pd.DataFrame([
        {
            'Unnamed: 0': livestock_covs.shape[0] + 1,
            'url': 'http://192.168.1.30:8333/gpw/livestock/gpw_livestock.pot.land_grassland.cropland.rules_p_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'type': 'temporal',
            'start_year': 2000,
            'end_year': 2022,
            'layername': 'gpw_livestock.pot.land_grassland.cropland.rules_p_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'spatial_res': '1km',
            'url_lan': 'http://192.168.1.30:8333/gpw/livestock/gpw_livestock.pot.land_grassland.cropland.rules_p_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'url_ib': 'http://192.168.49.40:8333/gpw/livestock/gpw_livestock.pot.land_grassland.cropland.rules_p_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'url_ext': 'http://s3.opengeohub.org:8333.30:8333/gpw/livestock/gpw_livestock.pot.land_grassland.cropland.rules_p_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'vrt': 'vrts/gpw_livestock.pot.land_grassland.cropland.rules_p_1km_{year}0101_{year}1231_go_epsg.4326_v1.vrt'
        },
        {
            'Unnamed: 0': livestock_covs.shape[0] + 2,
            'url': 'http://192.168.1.30:8333/gpw/livestock/gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'type': 'temporal',
            'start_year': 2000,
            'end_year': 2022,
            'layername': 'gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'spatial_res': '1km',
            'url_lan': 'http://192.168.1.30:8333/gpw/livestock/gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'url_ib': 'http://192.168.49.40:8333/gpw/livestock/gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'url_ext': 'http://s3.opengeohub.org:8333.30:8333/gpw/livestock/gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1.tif',
            'vrt': 'vrts/gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1.vrt'
        }
    ])
]).reset_index(drop=True)

In [2]:
from pathlib import Path

base_features = []

cmds = []
te = '-19927000 -6150000 20024000 7855000'
#Extent: (-17677000.000000, -6150000.000000) - (20024000.000000, 7855000.000000
#te = '-180 -90 180 90'

tr = '1000 1000'
#tr = '0.008 0.008'

t_srs = '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs'
#t_srs = 'EPSG:4326'

basedir = 'vrt'
#basedir = 'vrt_wgs84'

for id, row in livestock_covs.iterrows():
    r = 'near'
    if row['spatial_res'] == '10km':
        r = 'cubicspline'
    elif row['spatial_res'] != '1km':
        r = 'average'
    
    gi = str(30 + (int(id) % 15))
    url = row['url_ib'].replace('.40:8333',f'.{gi}:8333')
    basename = Path(url).stem
    
    if basename == 'gpw_livestock.systems_grassland.cropland.rules_c_1km_{year}0101_{year}1231_go_epsg.4326_v1':
        r = 'mode'
        print(basename, r)
        
    if row['type'] == 'static':
        if not Path(f'{wd}/{basedir}/{basename}.vrt').exists():
            cmds.append(
                f"gdalwarp -te {te} -t_srs '{t_srs}' -r {r} -tr {tr} /vsicurl/{url} {wd}/{basedir}/{basename}.vrt"
            )
    elif row['type'] == 'temporal':
        for y in range(int(row['start_year']), int(row['end_year'])+1):
            y_url = url.replace('{year}', str(y))
            y_basename = Path(y_url).stem
            if not Path(f'{wd}/{basedir}/{y_basename}.vrt').exists():
                cmds.append(
                    f"gdalwarp -te {te} -t_srs '{t_srs}' -r {r} -tr {tr} /vsicurl/{y_url} {wd}/{basedir}/{y_basename}.vrt"
                )
print(f"Cmd size: {len(cmds)}")

Cmd size: 23


In [18]:
for y in range(2000,2023):
    y_url = f'http://192.168.49.30:8333/gpw/arco/gpw_grassland_rf.savgol.bthr_c12_1km_{y}0101_{y}1231_go_epsg.4326_v1.tif'
    y_basename = Path(y_url).stem
    cmds.append(
        f"gdalwarp -te {te} -t_srs '{t_srs}' -r {r} -tr {tr} /vsicurl/{y_url} {wd}/{basedir}/{y_basename}.vrt"
    )

cmds.append(
    f"gdalwarp -te {te} -t_srs '{t_srs}' -r {r} -tr {tr} {wd}/gpw_grassland.mask_rf.savgol.bthr_max_1km_20000101_20221231_go_epsg.4326_v1.tif {wd}/{basedir}/gpw_grassland.mask_rf.savgol.bthr_max_1km_20000101_20221231_go_epsg.4326_v1.vrt"
)

cmds.append(
    f"gdalwarp -te {te} -t_srs '{t_srs}' -r {r} -tr {tr} /vsicurl/http://192.168.49.30:8333/gpw/arco/lcv_landmask_esacci.lc.l4_c_1km_s0..0cm_2000..2015_v1.0.tif {wd}/{basedir}/lcv_landmask_esacci.lc.l4_c_1km_s0..0cm_2000..2015_v1.0.vrt"
)

print(f"Cmd size: {len(cmds)}")

Cmd size: 25


In [7]:
f'{wd}/warp_vrt_2.in'

'/mnt/tupi/WRI/livestock_global_modeling/warp_vrt_2.in'

In [4]:
with open(f"{wd}/warp_vrt_2.in", "w") as txt_file:
    for line in cmds:
        txt_file.write(f"{line}\n")

### Check internal tiles

In [ ]:
from pathlib import Path
import rasterio

base_features = []

cmds = []
te = '-180 -90 180 90'
tr = '0.00800 0.00800'

for id, row in livestock_covs.iterrows():
    r = 'near'
    if row['spatial_res'] != '1km':
        r = 'average'
    
    gi = str(30 + (int(id) % 15))
    url = row['url_ib'].replace('.40:8333',f'.{gi}:8333')
    basename = Path(url).stem
    if row['type'] == 'static':
        src = rasterio.open(f'/vsicurl/{url}')
        _, window = next(src.block_windows(1))
        if (window.width == 1 or window.height == 1):
            print(f'{window}: {url}')
    elif row['type'] == 'temporal':
        for y in range(int(row['start_year']), int(row['end_year'])+1):
            y_url = url.replace('{year}', str(y))
            y_basename = Path(y_url).stem
            src = rasterio.open(f'/vsicurl/{y_url}')
            _, window = next(src.block_windows(1))
            if (window.width == 1 or window.height == 1):
                print(f'{window}: {y_url}')
print(f"Cmd size: {len(cmds)}")

## Base features

In [10]:
base_feat = []
for _, row in livestock_covs[livestock_covs['type'] == 'temporal'].sort_values('url_ib').iterrows():
    base_feat.append({
       'type': 'temporal',
       'name': Path(row['url_ib']).stem.replace('{year}','year'),
       'path': f'{wd}/vrt/' + Path(row['url_ib']).stem + '.vrt'
    })
for _, row in livestock_covs[livestock_covs['type'] == 'static'].sort_values('url_ib').iterrows():
    base_feat.append({
       'type': 'static',
       'name': Path(row['url_ib']).stem,
       'path': f'{wd}/vrt/' + Path(row['url_ib']).stem + '.vrt'
    })

for m in range(1,13):
    base_feat.append({
        'type': 'on-the-fly',
        'name': f'clm_lst_min.geom.temp_m_30m_s_m{m}',
        'path': ''
    })
    base_feat.append({
        'type': 'on-the-fly',
        'name': f'clm_lst_max.geom.temp_m_30m_s_m{m}',
        'path': ''
    })
    
base_feat.append({
        'type': 'on-the-fly',
        'name': f'bsf_veg_ndvi_mod13q1.v061_p50_250m_s0..0cm_v1',
        'path': ''
    })

base_feat = pd.DataFrame(base_feat)
base_feat.loc[:, 'idx'] = base_feat.index
base_feat.to_csv(f'{wd}/livestock_census_ard/features_livestock_all_zonal.csv')

## Tile system

In [25]:
from skmap.parallel import TilingProcessing
import rasterio

In [35]:
base_raster = '/mnt/tupi/WRI/livestock_global_modeling/vrt/gpw_grassland.mask_rf.savgol.bthr_max_1km_20000101_20221231_go_epsg.4326_v1.vrt'
#base_raster = 'http://192.168.49.35:8333/global/lc/lc_esa.worldcover_c_10m_s_20210101_20211231_go_epsg.4326_v20240901.tif'
ds = rasterio.open(base_raster)
tile_size = ds.transform[0] * 2250 #1024 * 4
#tile_size
tiling_system = TilingProcessing.generate_tiles(tile_size, ds.bounds, ds.crs, raster_layer_fn=base_raster)

In [42]:
tiling_system['x_size'] = (tiling_system['maxx'] - tiling_system['minx']) / ds.transform[0]
tiling_system['y_size'] = (tiling_system['maxy'] - tiling_system['miny']) / ds.transform[0]

In [43]:
tiling_system[tiling_system['raster_mode_count'] > 0].to_file(f'{wd}/gpw_livestock_tile_igh.gpkg', driver="GPKG")

### Converting grassland mask

In [ ]:
find ../../../livestock_global_modeling/cogs/ -name '*grassland*.tif' -exec echo echo gdal_translate -projwin -180 90 180 -90 -co COMPRESS=deflate -co TILED=true {} tmp/\`basename {}\` \; | bash | parallel -j10

### Area layer

In [21]:
import rasterio 
import numpy as np
base_raster = 'http://192.168.49.30:8333/gpw/arco/lcv_landmask_esacci.lc.l4_c_1km_s0..0cm_2000..2015_v1.0.tif'
ds = rasterio.open(base_raster)
data = ds.read(1, out_dtype='float64')
nodata = 2 #ds.nodatavals[0]

data[data == nodata] = np.nan
data[:] = (data * 0) + np.arange(0, data.shape[0] * data.shape[1], 1).reshape(data.shape)
data.shape

(22500, 45000)

In [3]:
#base_raster = 'http://192.168.1.30:8333/gpw/arco/gpw_grassland.mask_rf.savgol.bthr_max_1km_20000101_20221231_go_epsg.4326_v1.tif'

In [22]:
from pathlib import Path 
def _new_raster(base_raster, raster_file, data, window = None,
  dtype = None, nodata = None):

  if (not isinstance(raster_file, Path)):
    raster_file = Path(raster_file)

  raster_file.parent.mkdir(parents=True, exist_ok=True)

  if len(data.shape) < 3:
    data = np.stack([data], axis=2)

  x_size, y_size, nbands = data.shape

  with rasterio.open(base_raster, 'r') as base_raster:
    if dtype is None:
      dtype = base_raster.dtypes[0]

    if nodata is None:
      nodata = base_raster.nodata

    transform = base_raster.transform

    if window is not None:
      transform = rasterio.windows.transform(window, transform)

    return rasterio.open(raster_file, 'w',
            driver='GTiff',
            height=x_size,
            width=y_size,
            count=nbands,
            dtype=dtype,
            crs=base_raster.crs,
            compress='LZW',
            transform=transform,
            nodata=nodata)

with _new_raster(base_raster, 'seq_1km_wgs84.tif', data, None, dtype=np.int32, nodata=-1) as new_raster:
    band_dtype = new_raster.dtypes[0]
    data[:,:][np.isnan(data)] = new_raster.nodata
    new_raster.write(data.astype(band_dtype), indexes=1)

In [ ]:
gdal_polygonize.py -f FlatGeobuf seq_1km_wgs84.tif seq_1km_wgs84.fgb

In [ ]:
#ogrinfo -sql "SELECT (ST_Area(ST_Transform(geom, 0, null, '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs','+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs')) / 1000000) * 1000  as area_km2 FROM seq_1km_wgs84_area LIMIT 10" -dialect sqlite seq_1km_wgs84.fgb

ogr2ogr -sql "SELECT (ST_Area(ST_Transform(geometry, '+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs')) / 1000000) * 1000  as out, * FROM seq_1km_wgs84" -dialect sqlite seq_1km_wgs84_area.fgb seq_1km_wgs84.fgb

gdal_rasterize -ot Int16 -a_nodata -1 -co COMPRESS=deflate -co TILED=TRUE -tr 0.008 0.008 -a area_km2 seq_1km_wgs84_area.fgb seq_1km_wgs84_area_int.tif

gdal_edit.py -scale 0.001 seq_1km_wgs84_area.tif

gdalwarp -dstnodata -1 -co COMPRESS=deflate -co TILED=true -te -180 -72 180 90 seq_1km_wgs84_area.tif seq_1km_wgs84_area_2.tif

gdal_calc.py --overwrite --type "Int16"  --co COMPRESS=DEFLATE --co TILED=TRUE -A /vsicurl/http://192.168.49.40:8333/gpw/arco/gpw_cattle.total_eml_m_1km_s_20200101_20201231_go_epsg.4326_v20240809.tif -B seq_1km_wgs84_area_2.tif --calc="numpy.round((A*B)/10)" --outfile=gpw_cattle.total_eml_m_1km_s_20200101_20201231_go_epsg.4326_v20240809.tif

### Production system thresholds

In [1]:
import sys
sys.path.insert(0,'./scikit-map')

In [16]:
wd = '/mnt/tupi/WRI/livestock_global_modeling/livestock_systems'

raster_files = [
    f'{wd}/input/cropland_glad.potapov.et.al_p_1km_s_20150101_20151231_go_epsg.4326_v20240624.vrt',
    f'{wd}/input/gpw_cultiv.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1.vrt',
    f'{wd}/input/gpw_nat.semi.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1.vrt',
    f'{wd}/Global_Map_2015_wgs84_1km.tif'
]

In [17]:
raster_files

['/mnt/tupi/WRI/livestock_global_modeling/livestock_systems/input/cropland_glad.potapov.et.al_p_1km_s_20150101_20151231_go_epsg.4326_v20240624.vrt',
 '/mnt/tupi/WRI/livestock_global_modeling/livestock_systems/input/gpw_cultiv.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1.vrt',
 '/mnt/tupi/WRI/livestock_global_modeling/livestock_systems/input/gpw_nat.semi.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1.vrt',
 '/mnt/tupi/WRI/livestock_global_modeling/livestock_systems/Global_Map_2015_wgs84_1km.tif']

In [18]:
from skmap import io
raster_data = io.read_rasters_cpp(raster_files, verbose=True, n_jobs=96)
raster_data.shape

[23:00:24] Reading 4 layers using window=Window(col_off=0, row_off=0, width=45000, height=22500) and array=(4, 1012500000)
[23:01:31] End


(4, 1012500000)

In [19]:
import numpy as np
prod_system_class = np.unique(raster_data[3,:])
prod_system_class

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22.], dtype=float32)

In [20]:
import bottleneck as bn
from pathlib import Path

result = []

for c in prod_system_class:
    mask = (raster_data[3,:] == c)
    for i in range(0,3):
        result.append({
            'layer': Path(raster_files[i]).stem,
            'class': c,
            'mean': bn.nanmean(raster_data[i,:][mask]),
            'std': bn.nanstd(raster_data[i,:][mask])
        })
        print(result[-1])
        

{'layer': 'cropland_glad.potapov.et.al_p_1km_s_20150101_20151231_go_epsg.4326_v20240624', 'class': 0.0, 'mean': 0.2937079668045044, 'std': 4.774389743804932}
{'layer': 'gpw_cultiv.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1', 'class': 0.0, 'mean': 0.04937867820262909, 'std': 1.7079136371612549}
{'layer': 'gpw_nat.semi.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1', 'class': 0.0, 'mean': 0.19321051239967346, 'std': 3.9332025051116943}
{'layer': 'cropland_glad.potapov.et.al_p_1km_s_20150101_20151231_go_epsg.4326_v20240624', 'class': 1.0, 'mean': 67.0252456665039, 'std': 32.34263229370117}
{'layer': 'gpw_cultiv.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1', 'class': 1.0, 'mean': 9.534008026123047, 'std': 20.196483612060547}
{'layer': 'gpw_nat.semi.grassland_rf.savgol.bthr.mode5_p_1km_20150101_20151231_go_epsg.4326_v1', 'class': 1.0, 'mean': 8.239190101623535, 'std': 23.935331344604492}
{'layer': 'cropland_glad.p

In [25]:
class_remap = {
  0: 'Cropland - Low Intensity',
  1: 'Cropland - Medium Intensity',
  2: 'Cropland - High Intensity',
  3: 'Mosaic Crop and Grass - Low Intensity',
  4: 'Mosaic Crop and Grass - Med Intensity',
  5: 'Mosaic Crop and Grass - High Intensity',
  6: 'Mosaic Crop and Forest - Low Intensity',
  7: 'Mosaic Crop and Forest - Med Intensity',
  8: 'Mosaic Crop and Forest - High Intensity',
  9: 'Dense Forest',
  10: 'Open Forest',
  11: 'Mosaic Grass and Forest',
  12: 'Mosaic Grass and Bare',
  13: 'Grassland',
  14: 'Grazing Grassland',
  15: 'Grazing Grass and Forest Mosaic',
  16: 'Shrubland',
  17: 'Peri-urban',
  18: 'Dense Urban',
  19: 'Plantation Forest',
  20: 'Wetland',
  21: 'Bare',
  22: 'Snow/Ice'
}

In [27]:
import pandas as pd 
result = pd.DataFrame(result)
result['class'] = result['class'].astype('int').map(class_remap)

In [30]:
result.to_csv('production_systems_cropland_grassland_zonal.csv')

Livestock mostly supported by natural/semi-natural grassland systems:
natural/semi-natural grassland > cultivated grassland area
0%—19% of cropland area

Livestock mostly supported by cultivated grassland systems
cultivated grassland > natural/semi-natural grassland area
0%—19% of cropland area

Livestock supported by mosaic cropland & grassland systems
grassland area > 0% 
19%—37% of cropland area

Livestock supported by cropland systems:
36%—71% of cropland area

Mosaic Crop and Grass - Low Intensity: 19
Mosaic Crop and Grass - High Intensity: 37
Cropland - High Intensity: 71

In [41]:
result[result['layer'].str.contains('crop')].sort_values('mean')

,layer,class,mean,std
66,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Snow/Ice,0.004848,0.270566
0,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Cropland - Low Intensity,0.293708,4.774390
63,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Bare,0.330206,3.931257
36,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Mosaic Grass and Bare,0.562868,5.302999
48,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Shrubland,0.567945,4.878347
27,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Dense Forest,0.684600,5.062541
39,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Grassland,1.240195,7.868424
33,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Mosaic Grass and Forest,1.788719,8.937921
30,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Open Forest,2.073226,9.035225
45,cropland_glad.potapov.et.al_p_1km_s_20150101_2...,Grazing Grass and Forest Mosaic,3.917636,13.111601
